In [1]:
%pip install -q --upgrade llama-index-llms-together llama-index llama-index-embeddings-jinaai llama-index-vector-stores-chroma docx2txt transformers torch "numpy<2" gTTS datasets sentencepiece backports.lzma pylzma openai SpeechRecognition pyaudio


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### What is Retrieval Augmented Generation (RAG)?
RAG is a technique that enhances language models by combining them with a retrieval system. It allows the model to access and utilize external knowledge when generating responses.

The process typically involves:
#### Indexing a large corpus of documents

In [ ]:
import os
os.environ["TOGETHER_API_KEY"] =
os.environ["JINAAI_API_KEY"] = 
os.environ["OPENAI_API_KEY"] = 

### Call LLM

In [3]:
from llama_index.llms.together import TogetherLLM

llm = TogetherLLM(
    api_key=os.environ["TOGETHER_API_KEY"],
    model="meta-llama/Llama-Vision-Free",
)


llm_response = llm.complete("Tell me a joke")

print(llm_response)

/Users/abubakarsiddique/.pyenv/versions/3.10.12/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Here's one:

What do you call a fake noodle?

An impasta.


### LLM Messages

In [4]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="What is your name"),
]
resp = llm.chat(messages)
print(resp)

assistant: Me hearty! Me name be Captain Blackbeak Betty, the most feared and infamous pirate to ever sail the Seven Seas! Me be a swashbucklin' scallywag with a heart o' gold and a spirit o' adventure. Me ship, the "Maverick's Revenge", be me home, and me trusty cutlass, "Betsy", be me best mate.

Me be a pirate of great renown, with a reputation for bein' the most cunning and clever on the high seas. Me have battled the Royal Navy, outwitted the scurvy dogs, and discovered hidden treasure beyond yer wildest dreams!

So, what be bringin' ye to these fair waters? Are ye lookin' to join me crew and sail the seas with the greatest pirate of all time? Or be ye just lookin' for a bit o' adventure and a taste o' the pirate life? Either way, ye be in luck, matey, 'cause Captain Blackbeak Betty be here to guide ye through the choppiest o' seas and the most treacherous o' waters!


In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from IPython.display import Markdown, display
import chromadb

documents = SimpleDirectoryReader("./docs/").load_data()

print(len(documents))



2


## Splitter

In [6]:
from llama_index.core import PromptHelper
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings

Settings.node_parser = SentenceSplitter(chunk_size=300, chunk_overlap=30)

### Embeddings

In [7]:
from llama_index.embeddings.jinaai import JinaEmbedding

embed_model = JinaEmbedding(
    api_key=os.environ["JINAAI_API_KEY"],
    model="jina-embeddings-v3",
)

## Set Settings

In [8]:
Settings.llm = llm
Settings.embed_model = embed_model
Settings.num_output = 512
Settings.context_window = 4096
Settings.chunk_overlap_ratio = 0.1

### Create and persist Chroma vector store

In [9]:
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("my_collection")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    settings = Settings
    )

## Chat Engine English

In [10]:
query_engine = index.as_chat_engine()
response = query_engine.chat("When was TechNova Solutions founded?")
print(response)

TechNova Solutions was founded in 2015.


In [11]:
query_engine = index.as_chat_engine()
response = query_engine.chat("What is the website of TechNova Solutions?")
print(response)

The website of TechNova Solutions is www.technovasolutions.com.


## Voice RAG Chatbot

In [17]:
from openai import OpenAI
import speech_recognition as sr
from IPython.display import Audio
client = OpenAI(api_key = os.environ["OPENAI_API_KEY"])

### Speech to text using microphone via ffmpeg and OpenAI Whisper

In [18]:
def transcribe():
    ''' listen to the users speech to determine what the next step should be'''
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Start speaking...")
        audio = r.listen(source)
        print ("Processing...")
    client = OpenAI()
    with open("microphone-results.wav", "wb") as f:
        f.write(audio.get_wav_data())

    with open("microphone-results.wav", "rb") as f:
        transcript = client.audio.transcriptions.create(
        model="whisper-1", 
        file=f,
        response_format="text",
        )
        print("Question: ",transcript)
    return transcript

### Text to Speech using OPENAI TTS model

In [14]:
aud_file = "text_to_speech.mp3"
def synthesise(text):
    response = client.audio.speech.create(
    model="tts-1",
    voice="onyx",
    input=text
)

    response.stream_to_file(aud_file)

#### Question Answering using both Transcribe and Synthesise Functions.

In [19]:
query_engine = index.as_chat_engine()
response = query_engine.chat(transcribe())
print("Answer :",response)
synthesise(response.response)
Audio(aud_file, rate=16000)

Start speaking...
Processing...
Question:  When was Technova Solutions founded?

Answer : Technova Solutions was founded in 2015.


/var/folders/5l/5xwv8y396kd9m_fw313f56g40000gn/T/ipykernel_4989/3842257161.py:9: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(aud_file)


In [20]:
query_engine = index.as_chat_engine()
response = query_engine.chat(transcribe())
print("Answer :",response)
audio = synthesise(response.response)
Audio(aud_file, rate=16000)

Start speaking...
Processing...
Question:  What is the website of Technova Solution?

Answer : The website of Technova Solution is www.technovasolutions.com.


/var/folders/5l/5xwv8y396kd9m_fw313f56g40000gn/T/ipykernel_4989/3842257161.py:9: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(aud_file)
